In [1]:
from opentree import OT

In [2]:
fam_fi=open("Family_tree_GF.txt").readlines()

In [3]:
fam_spps = set()
for lin in fam_fi:
    lii = lin.strip()
    fam_spps.add(lii)

In [4]:
len(fam_spps)

916

In [5]:
fam_dict = dict()
for fam_name in fam_spps:
    if fam_name not in fam_dict:
        print('.')
        ott_id = OT.get_ottid_from_name(fam_name)
        fam_dict[fam_name] = ott_id 

In [134]:
fam_ott_ids = set()
no_match = set()
for fam_name in fam_dict:
    ott_id = fam_dict[fam_name]
    if ott_id:
        fam_ott_ids.add(ott_id)
    else:
        print("No ott_id for {}".format(fam_name))

No ott_id for Clostridiales Family XVII. Incertae Sedis
No ott_id for Rep1
No ott_id for Clostridiales Family XII. Incertae Sedis
No ott_id for Clostridiales Family XIII. Incertae Sedis


In [12]:
len(fam_dict)

916

In [13]:
len(fam_ott_ids)

910

In [16]:
try:
    fam_ott_ids.remove(None)
except:
    pass

In [174]:
output = OT.synth_induced_tree(ott_ids=list(fam_ott_ids),  label_format='name_and_id')

In [175]:
import copy
local_tree = copy.deepcopy(output.tree)

In [122]:
def remove_problem_characters(tree, prob_char = "():#", replace_w = '?'):
    problem_characters = set(prob_char)
    for node in tree:
        if node.taxon:
            for char in problem_characters:
                node.taxon.label = node.taxon.label.replace(char,replace_w)
        elif node.label:
            for char in problem_characters:
                node.label = node.label.replace(char,replace_w)
    return None

In [159]:
def quote_problem_characters(tree, prob_char = "():#", replace_w = '?'):
    problem_characters = set(prob_char)
    for node in tree:
        if node.taxon:
            for char in problem_characters:
                node.taxon.label = '{}{}{}'.format("'",node.taxon.label, "'")
        elif node.label:
            for char in problem_characters:
                node.label = '{}{}{}'.format("*",node.label, "*")
    return None

In [176]:
quote_problem_characters(local_tree)
treefile = "FAM_cal_eDNA_quot.tre"
local_tree.write(path = treefile, schema = "newick")

In [104]:
treestr = output.tree.as_string(schema = "newick")
":" in treestr

False

In [177]:
remove_problem_characters(output.tree)

In [178]:
rev_broken =  {v: k for k, v in broken.items()}
rev_fam =  {v: k for k, v in fam_dict.items()}
for node in output.tree:
    if node.taxon:
        if node.taxon.label in rev_broken:
            fam = rev_fam[int(rev_broken[node.taxon.label].strip('ott'))]
            node.taxon.label = "{} broken - MRCA {}".format(fam, node.taxon.label)
    elif node.label:
         if node.label in rev_broken:
            fam = rev_fam[int(rev_broken[node.label].strip('ott'))]
            node.label = "{} broken - MRCA {}".format(fam, node.label)

In [179]:
treefile = "FAM_cal_eDNA_mrca.tre"
output.tree.write(path = treefile, schema = "newick")

In [130]:
synth_leaf_ott_ids = set()
synth_node_ids = set()
#for taxon in output.tree.taxon_namespace:
for node in output.tree:
    if node.taxon:
        synth_leaf_ott_ids.add(node.taxon.label.split()[-1])
    if node.label:
        synth_node_ids.add(node.label.split()[-1])


In [136]:
broken = output.response_dict['broken']

In [133]:
outfi = open('fam_info_synth.csv','w')
outfi.write("fam_name, ott_id, broken_taxon, mrca_of_descendents, in_synth_tree\n")
for fam_name in fam_dict:
    ott_id = fam_dict[fam_name]
    brok = "-"
    mrca = "-"
    in_synth_tree = 'False'
    if ott_id != 'None':
        ott_str = "ott{}".format(ott_id)
        if ott_str in broken:
            brok = "T"
            mrca = broken[ott_str].strip()
            if mrca in synth_leaf_ott_ids:
                in_synth_tree = 'Leaf'
            elif mrca in synth_node_ids:
                in_synth_tree = 'Node'
        if ott_str in synth_leaf_ott_ids:
            in_synth_tree = 'Leaf'
        elif ott_str in synth_node_ids:
            in_synth_tree = 'Node'
            
    outfi.write("{},{},{},{},{}\n".format(fam_name, ott_id, brok, mrca, in_synth_tree))
outfi.close()

In [180]:
missing_fams = set()
for fam_name in fam_dict:
    ott_id = fam_dict[fam_name]
    brok = "-"
    mrca = "-"
    in_synth_tree = 'False'
    if ott_id != 'None':
        ott_str = "ott{}".format(ott_id)
        if ott_str in broken:
            brok = "T"
            mrca = broken[ott_str].strip()
            if mrca in synth_leaf_ott_ids:
                in_synth_tree = 'Leaf'
            elif mrca in synth_node_ids:
                in_synth_tree = 'Node'
        if ott_str in synth_leaf_ott_ids:
            in_synth_tree = 'Leaf'
        elif ott_str in synth_node_ids:
            in_synth_tree = 'Node'
    if in_synth_tree == 'False':
        missing_fams.add(fam_name)

In [392]:
fi = open("all_asv_transects_for_EJMT.txt")
spp_reps = {}
for fam in missing_fams:
    spp_reps[fam] = set()
for lin in fi:
    lii = lin.split(';')
    while("" in lii) : 
        lii.remove("") 
    for htax in lii:
        if htax in missing_fams:
            spp_reps[htax].add(lii[-1].strip())

In [393]:
spp_reps['Oscillatoriaceae']

{'',
 'Lyngbya aestuarii',
 'Lyngbya polychroa',
 'Oscillatoria acuminata',
 'Oscillatoria tenuis',
 'Phormidium autumnale',
 'Phormidium chlorinum',
 'Phormidium laetevirens',
 'Phormidium sp. B-Tom',
 'Phormidium sp. PUPCCC 118.2'}

In [394]:
#spp_ids = {}
for fam in spp_reps:
    for spp_name in spp_reps[fam]:
        if spp_name not in spp_ids:
            ott_id = OT.get_ottid_from_name(spp_name)
            spp_ids[spp_name] = ott_id

In [395]:
fam_mrca = {}
for fam in spp_reps:
    ott_ids = set([spp_ids[spp] for spp in spp_reps[fam]])
    if None in ott_ids:
        ott_ids.remove(None)
    if len(ott_ids) < 1:
        print(fam)
        fam_mrca[fam] = None
    else:
        ret = 0
        while ret == 0:
            assert(ott_ids)
            call_record = OT.ws.tree_of_life_mrca(ott_ids=list(ott_ids))
            if 'unknown' in call_record.response_dict:
                for key in call_record.response_dict['unknown']:
                    ott_ids.remove(int(key.strip('ott')))
            elif 'mrca' in call_record.response_dict:
                ret = 1
                fam_mrca[fam] = call_record.response_dict['mrca']['node_id']              
            if len(ott_ids) < 1:
                print(fam)
                msgtemplate = 'Call to tree_of_life/mrca failed as all ids were pruned'
                print(msgtemplate)
                fam_mrca[fam] = None
                ret = 1
    

Hemiaulaceae
Rep1
Malasseziaceae
Call to tree_of_life/mrca failed as all ids were pruned
Phaeocystaceae
Labilitrichaceae
Call to tree_of_life/mrca failed as all ids were pruned
Marinifilaceae
Call to tree_of_life/mrca failed as all ids were pruned
Pseudanabaenaceae
Call to tree_of_life/mrca failed as all ids were pruned
Phyllophoraceae
Call to tree_of_life/mrca failed as all ids were pruned
Microbulbiferaceae
Minutisphaeraceae
Call to tree_of_life/mrca failed as all ids were pruned
Nitrosopumilaceae
Call to tree_of_life/mrca failed as all ids were pruned
Ancyromonadidae
Call to tree_of_life/mrca failed as all ids were pruned
Clostridiales Family XIII. Incertae Sedis
Call to tree_of_life/mrca failed as all ids were pruned
Pseudeurotiaceae
Call to tree_of_life/mrca failed as all ids were pruned
Coleofasciculaceae
Call to tree_of_life/mrca failed as all ids were pruned
Tetrahymenidae
Pseudoperisporiaceae
Call to tree_of_life/mrca failed as all ids were pruned
Sanchytriaceae
Call to tree_o

In [396]:
fam_mrca_ids = [fam_mrca[fam] for fam in fam_mrca]

In [457]:
fam_ott_ids = ['ott{}'.format(idn) for idn in fam_ott_ids]

In [458]:
ids_plus = fam_mrca_ids + fam_ott_ids
ids_plus = set(ids_plus)
ids_plus.remove(None)

In [459]:
outputplus = OT.synth_induced_tree(node_ids = list(ids_plus), label_format='name_and_id')

In [460]:
outputplus.response_dict

{'broken': {'ott1021896': 'mrcaott23276ott23291',
  'ott1023962': 'mrcaott14541ott124912',
  'ott1024716': 'mrcaott6117ott105428',
  'ott1025541': 'mrcaott2556ott3381',
  'ott1025751': 'mrcaott42379ott140961',
  'ott1033801': 'mrcaott8800ott405457',
  'ott1038708': 'ott971709',
  'ott104185': 'ott971709',
  'ott1047456': 'ott51652',
  'ott1050974': 'mrcaott1403ott8433',
  'ott1051950': 'ott971709',
  'ott1085835': 'mrcaott23276ott60764',
  'ott1099024': 'ott971709',
  'ott112540': 'mrcaott5864ott7040',
  'ott121698': 'ott51652',
  'ott122715': 'mrcaott50ott184',
  'ott126141': 'mrcaott10354ott17309',
  'ott133074': 'ott183800',
  'ott134473': 'mrcaott1093ott2355',
  'ott137782': 'mrcaott103ott1562',
  'ott139533': 'mrcaott1492ott5822',
  'ott146473': 'ott822495',
  'ott147029': 'ott5261925',
  'ott15312': 'mrcaott15308ott15310',
  'ott158309': 'mrcaott89ott26838',
  'ott160603': 'ott767314',
  'ott161886': 'ott588758',
  'ott162651': 'ott1012685',
  'ott16694': 'mrcaott725ott1456',
  '

In [463]:
tmp_tree = copy.deepcopy(outputplus.tree)

In [464]:
rev_broken =  {v: k for k, v in broken.items()}
rev_fam =  {v: k for k, v in fam_dict.items()}
rev_mrca =  {v: k for k, v in fam_mrca.items()}

added_fams = set()
labels = set()
for node in tmp_tree:
    if node.taxon:
        labels.add(node.taxon.label )
        if node.taxon.label in rev_broken:
            fam = rev_fam[int(rev_broken[node.taxon.label].strip('ott'))]
            node.taxon.label = "{} broken - MRCA {}".format(fam, node.taxon.label)
        elif node.taxon.label.split()[-1] in rev_mrca:
            added_fams.add(node.taxon.label.split()[-1])
            fam = rev_mrca[node.taxon.label.split()[-1]]
            node.taxon.label = "{} missing - MRCA {}".format(fam, node.taxon.label)
    elif node.label:
        labels.add(node.label)
        if node.label in rev_broken:
            fam = rev_fam[int(rev_broken[node.label].strip('ott'))]
            node.label = "{} broken - MRCA {}".format(fam, node.label)
        elif node.label.split()[-1] in rev_mrca:
            added_fams.add(node.label.split()[-1])
            fam = rev_mrca[node.label.split()[-1]]
            node.label = "{} missing - MRCA {}".format(fam, node.label)

In [465]:
len(added_fams)

184

In [466]:
len(rev_mrca)

191

In [467]:
remove_problem_characters(tmp_tree)

In [468]:
treefile = "FAM_cal_eDNA_addedFams.tre"
tmp_tree.write(path = treefile, schema = "newick")

In [469]:
synth_leaf_ott_ids = set()
synth_node_ids = set()
#for taxon in output.tree.taxon_namespace:
for node in outputplus.tree:
    if node.taxon:
        synth_leaf_ott_ids.add(node.taxon.label.split()[-1])
    if node.label:
        synth_node_ids.add(node.label.split()[-1])


In [470]:
outfi = open('fam_info_synth_plus.csv','w')
outfi.write("fam_name, ott_id, broken_taxon, mrca_of_descendents, in_synth_tree, selected_descendents\n")
for fam_name in fam_dict:
    ott_id = fam_dict[fam_name]
    brok = "-"
    mrca = "-"
    in_synth_tree = 'False'
    selected_descendents = "-"
    if ott_id != 'None':
        ott_str = "ott{}".format(ott_id)
        if ott_str in broken:
            brok = "T"
            mrca = broken[ott_str].strip()
            if mrca in synth_leaf_ott_ids:
                in_synth_tree = 'Leaf'
            elif mrca in synth_node_ids:
                in_synth_tree = 'Node'
        if fam_name in fam_mrca:
            brok = "missing"
            mrca = fam_mrca[fam_name]
            if mrca in synth_leaf_ott_ids:
                in_synth_tree = 'Leaf'
            elif mrca in synth_node_ids:
                in_synth_tree = 'Node'
            selected_descendents = ";".join(list(set(spp_reps[fam_name])))
        if ott_str in synth_leaf_ott_ids:
            in_synth_tree = 'Leaf'
        elif ott_str in synth_node_ids:
            in_synth_tree = 'Node'
            
    outfi.write("{},{},{},{},{},{}\n".format(fam_name, ott_id, brok, mrca, in_synth_tree,selected_descendents))
outfi.close()

In [471]:
len(fam_dict.keys())

916

In [489]:
rev_broken

{'mrcaott23276ott23291': ['ott1021896'],
 'mrcaott14541ott124912': ['ott1023962'],
 'mrcaott6117ott105428': ['ott1024716'],
 'mrcaott2556ott3381': ['ott1025541'],
 'mrcaott42379ott140961': ['ott1025751'],
 'mrcaott8800ott405457': ['ott1033801'],
 'ott971709': ['ott1038708',
  'ott104185',
  'ott1051950',
  'ott1099024',
  'ott168443',
  'ott419068',
  'ott419069',
  'ott5345481',
  'ott728370',
  'ott826618',
  'ott827790',
  'ott969668',
  'ott989246'],
 'ott51652': ['ott1047456', 'ott121698', 'ott538149'],
 'mrcaott1403ott8433': ['ott1050974'],
 'mrcaott23276ott60764': ['ott1085835'],
 'mrcaott5864ott7040': ['ott112540'],
 'mrcaott50ott184': ['ott122715'],
 'mrcaott10354ott17309': ['ott126141'],
 'ott183800': ['ott133074'],
 'mrcaott1093ott2355': ['ott134473'],
 'mrcaott103ott1562': ['ott137782', 'ott441950'],
 'mrcaott1492ott5822': ['ott139533'],
 'ott822495': ['ott146473'],
 'ott5261925': ['ott147029'],
 'mrcaott15308ott15310': ['ott15312'],
 'mrcaott89ott26838': ['ott158309'],
 'o

In [486]:
clean_tree = copy.deepcopy(tmp_tree)

In [488]:
got_fam = set()
for node in clean_tree:
    if node.taxon:
        if node.taxon.label.split()[0] in fam_dict.keys():
            node.taxon.label = node.taxon.label.split()[0]
            got_fam.add(node.taxon.label.split()[0])
        else:
            ottid = node.taxon.label.split()[-1]
            if ottid.startswith('ott'):
                if int(ottid.strip('ott')) in rev_fam:
                    node.taxon.label = rev_fam[int(ottid.strip('ott'))]
                elif ottid in rev_broken:
                    node.taxon.label = rev_fam[int(rev_broken[ottid].strip('ott'))]
                elif ottid in rev_mrca:
                    node.taxon.label = rev_mrca[ottid]
                else:
                    print(node.taxon.label)
            elif node.taxon.label.split()[-1].startswith('mrca'):
                if ottid in rev_mrca:
                    node.taxon.label = rev_mrca[ottid]
                elif ottid in rev_broken:
                    node.taxon.label = rev_fam[int(rev_broken[ottid].strip('ott'))]
                else:
                    print(node.taxon.label)   
    elif node.label:
        if node.label.split()[0] in fam_dict.keys():
            node.label = node.label.split()[0]
            got_fam.add(node.label.split()[0])
        else:
            node.label = ''
    
    


AttributeError: 'list' object has no attribute 'strip'

In [485]:
treefile = "family_clean.tre"
clean_tree.write(path = treefile, schema = "newick")

In [479]:
# Don't forget to cite your friendly phylogeneticists!
studies = outputplus.response_dict['supporting_studies']
cites = OT.get_citations(studies) #this can be a bit slow
fi = open("famcites.txt","w")
fi.write(cites)
fi.close()

In [515]:
all_ids_to_fam= dict()

for fam_name in fam_dict:
    ottstr = "ott{}".format(fam_dict[fam_name])
    if ottstr not in rev_fam:
        all_ids_to_fam[ottstr] = set()
    all_ids_to_fam[ottstr].add(fam_name)


In [516]:
for brok_ottid in broken:
    if brok_ottid not in all_ids_to_fam:
        print("oop")
        print(brok_ottid)
    cor_ottid = broken[brok_ottid]
    all_ids_to_fam[cor_ottid]= all_ids_to_fam[brok_ottid]

In [517]:
for fam_name in fam_mrca:
    mrca = fam_mrca[fam_name]
    if mrca not in all_ids_to_fam:
        all_ids_to_fam[mrca] = set()
    all_ids_to_fam[mrca].add(fam_name)
    
del all_ids_to_fam[None]

In [534]:
double_tree = copy.deepcopy(tmp_tree)

In [535]:
for node in double_tree:
    if node.taxon:
        ottstr = node.taxon.label.split()[-1]
        famlist = all_ids_to_fam[ottstr]
        if len(famlist) == 1:
            node.taxon.label = list(famlist)[0]
        elif len(famlist) > 1:
            node.taxon.label = '-'.join(list(famlist))
        else:
            print(node.taxon.label)
    if node.label:
        ottstr = node.label.split()[-1]
        if ottstr in all_ids_to_fam:
            famlist = all_ids_to_fam[ottstr]
            if len(famlist) == 1:
                node.label = list(famlist)[0]
            elif len(famlist) > 1:
                node.label = '-'.join(list(famlist))+ottstr

AttributeError: 'NoneType' object has no attribute 'label'

In [536]:
treefile = "family_double_clean.tre"
double_tree.write(path = treefile, schema = "newick")

In [526]:
node.taxon.label

'Nitrososphaeraceae'